In [10]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import Ollama
from langchain.text_splitter import TokenTextSplitter
import glob

Load the document.

In [17]:
# loader = PyPDFLoader("docs/*.pdf")
# pages = loader.load()

loaders = [
    PyPDFLoader(path) for path in glob.glob("docs/*.pdf")
]
pages = []
for loader in loaders:
    pages.extend(loader.load())

In [18]:
len(pages) # there are more docs than loaders because each pdf has multiple pages

344

In [19]:
len(loaders)

33

Text splitting.

In [20]:
text_splitter = TokenTextSplitter(chunk_size=100, chunk_overlap=25)
docs = text_splitter.split_documents(pages)


Create the embeddings and the vectorstore.

In [21]:
model_name = "BAAI/bge-small-en"
model_kwargs = {"device": "cpu"}
encode_kwargs = {"normalize_embeddings": True}
hf = HuggingFaceBgeEmbeddings(
    model_name=model_name, model_kwargs=model_kwargs, encode_kwargs=encode_kwargs
)

persist_directory = 'docs/chroma/'
!rm -rf docs/chroma  # remove old database files if any

vectordb = Chroma.from_documents( # had an error previuously, downgraded to chromadb version 0.4.3 using command: pip install chromadb==0.4.3. See https://github.com/zylon-ai/private-gpt/issues/1012
    documents=docs,
    embedding=hf,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

/Users/cameliadanielabrumar/anaconda3/envs/llama-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/cameliadanielabrumar/anaconda3/envs/llama-env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


8834


Import the local llm.

In [32]:
llm = Ollama(model="llama3:8b", temperature=0.0) # setting temperature to 0.0 to get deterministic results, with low variability and gives us highest fidelity and reliable answers

Question answering with prompts and memory.

In [37]:
ConversationalRetrievalChain.__doc__

'[*Deprecated*] Chain for having a conversation based on retrieved documents.\n\n    This class is deprecated. See below for an example implementation using\n    `create_retrieval_chain`. Additional walkthroughs can be found at\n    https://python.langchain.com/docs/use_cases/question_answering/chat_history\n\n        .. code-block:: python\n\n            from langchain.chains import (\n                create_history_aware_retriever,\n                create_retrieval_chain,\n            )\n            from langchain.chains.combine_documents import create_stuff_documents_chain\n            from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder\n            from langchain_openai import ChatOpenAI\n\n\n            retriever = ...  # Your retriever\n\n            llm = ChatOpenAI()\n\n            # Contextualize question\n            contextualize_q_system_prompt = (\n                "Given a chat history and the latest user question "\n                "which might refe

In [33]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True # returns the chat history as a list of messages as opposed to a string, more details on memory in the previous class from this guy
)

retriever=vectordb.as_retriever(search_type="mmr")
qa = ConversationalRetrievalChain.from_llm( # adds something more on top of the retrieval chain, not just memory. It adds a step that takes the history and the new question and condenses it into a standalone question that's passed to the vectorstore to look up relevant documents.               
    llm,
    retriever=retriever,
    memory=memory
)

Chatting with the llm.

In [34]:
question = "What are the types of decisions in the typology of decision-making tasks?"
result = qa({"question": question})
result['answer']

'According to the provided context, the typology of decision-making tasks consists of three tasks:\n\n1. CHOOSE\n2. ACTIVATE\n3. CREATE\n\nThese tasks represent specific and distinct decision problems.'

In [35]:
question = "What are the 3 types again??"
result = qa({"question": question})
result['answer']

'According to the provided context, the three types of decisions in the typology of decision-making tasks are:\n\n1. CHOOSE\n2. ACTIVATE\n3. CREATE\n\nThese tasks were derived from scientific literature and represent specific and distinct decision problems.'

In [36]:
question = "Give me an example of a choose task."
result = qa({"question": question})
result['answer']

'Based on the provided context, an example of a "CHOOSE" task is not explicitly mentioned. However, it can be inferred that a "CHOOSE" task assesses options and outputs a subset deemed optimal or best.\n\nIn the given examples, the top predictions of both options represent "Read Email" but have different probabilities. This could be considered as an example of a decision-making problem where options are evaluated to determine the best choice. However, it is not explicitly labeled as a "CHOOSE" task.\n\nTherefore, I don\'t know the specific answer to this question based on the provided context.'

In [27]:
question = "Give me a real-world example of that task found in one of the provided documents."
result = qa({"question": question})
result['answer']

'According to the context, an example of a CHOOSE task is buying a car. In this scenario, we have four cars from which we can choose. We could score the cars on their features relative to the other cars in the set, and select the car with the top score.'

In [38]:
retriever.retrieved_docs

AttributeError: 'VectorStoreRetriever' object has no attribute 'retrieved_docs'